<a href="https://colab.research.google.com/github/marquesarthur/vanilla-bert-vs-huggingface/blob/main/hugging_face_keras_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on 



1.   https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
2.   https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/
3.   https://huggingface.co/transformers/training.html#fine-tuning-with-keras




**problem statement:**


*   a developer has to inspect an **artifact X**
*   Within the artifact, only a portion of the text is relevant to **input task Y**
*   We ought to build a model that establishes relationships between **Y** and **sentences x ∈ X** 
*  The model must determine: **is x relevant to task Y**




<br>

___

*Example of a task and an annotated artifact:*

<br>

[<img src="https://i.imgur.com/Zj1317H.jpg">](https://i.imgur.com/Zj1317H.jpg)




* The coloured sentences are sentences annotated as relevant to the input task. 
* The warmer the color, the more annotators selected that portion of the text. 
* For simplicity, we process the data and used sentences 

<br>

___

*Ultimately, our data is a tuple representing:*


*   **text** = artifact sentence

*   **question** = task description

*   **source** = URL of the artifact

*   **category_index** = whether sentence is relevant [or not] for the input task

*   **weights** = number of participants who annotated sentence as relevant


<br>

___



In [1]:
# @title Install dependencies

# !pip install transformers
# %tensorflow_version 2.x

In [2]:
# !pip install scikit-learn tqdm pandas python-Levenshtein path colorama matplotlib seaborn

In [3]:
# !pip install python-Levenshtein

In [4]:
# @title Download git repo
# !git clone https://github.com/marquesarthur/vanilla-bert-vs-huggingface.git

In [5]:
# %cd vanilla-bert-vs-huggingface
# !git pull
# !ls -l

In [6]:
# @title Import data as JSON
import itertools
import json
import logging
import os
import sys
import random
from pathlib import Path

from Levenshtein import ratio
from colorama import Fore, Style

logger = logging.getLogger()
logger.level = logging.DEBUG
stream_handler = logging.StreamHandler(sys.stdout)
logger.addHandler(stream_handler)

from ds_android import get_input_for_BERT

raw_data = get_input_for_BERT()

print('Sample entry from data:')
print(json.dumps(raw_data[0], indent=4, sort_keys=True))

39 129  https://developer.android.com/training/permissions/requesting
14 21  https://stackoverflow.com/questions/5233543
4 34  https://github.com/morenoh149/react-native-contacts/issues/516
27 63  https://guides.codepath.com/android/Understanding-App-Permissions
9 161  https://www.avg.com/en/signal/guide-to-android-app-permissions-how-to-use-them-smartly
9 15  https://developer.android.com/training/volley/request
14 65  https://stackoverflow.com/questions/28504524
20 59  https://medium.com/@JasonCromer/android-asynctask-http-request-tutorial-6b429d833e28
5 97  https://www.twilio.com/blog/5-ways-to-make-http-requests-in-java
4 12  https://stackoverflow.com/questions/33241952
6 33  https://github.com/realm/realm-java/issues/776
3 17  https://stackoverflow.com/questions/8712652
8 59  https://dzone.com/articles/android-rotate-and-scale
5 470  https://developer.android.com/reference/android/widget/TextView
7 11  https://stackoverflow.com/questions/19025301
8 95  https://docs.oracle.com/java

4 54  https://developer.android.com/training/gestures/scroll
4 16  https://stackoverflow.com/questions/39588322
20 196  https://developer.android.com/training/dependency-injection/dagger-android
6 44  https://stackoverflow.com/questions/57235136
24 121  https://guides.codepath.com/android/dependency-injection-with-dagger-2
Sample entry from data:
{
    "category_index": 1,
    "question": "Permission Denial when trying to access contacts in Android",
    "source": "https://developer.android.com/training/permissions/requesting",
    "text": "Every Android app runs in a limited-access sandbox.",
    "weights": 1
}


In [7]:
from collections import Counter, defaultdict

cnt = Counter([d['category_index'] for d in raw_data])

total = sum(cnt.values())

labels_cnt = [cnt[0] / float(total), cnt[1] / float(total)]
print('label distribution')
print('')
print('not-relevant -- {:.0f}%'.format(labels_cnt[0] * 100))
print('RELEVANT ------ {:.0f}%'.format(labels_cnt[1] * 100))

label distribution

not-relevant -- 88%
RELEVANT ------ 12%


In [8]:
seframes = {}
with open('seframes.json') as input_file:
    seframes = json.load(input_file)

In [9]:
def has_meaningful_frame(text):    
    meaning_frames = [
        'Temporal_collocation', 'Execution', 'Using', 'Intentionally_act',
        'Being_obligated', 'Likelihood', 'Causation', 'Required_event',
        'Desiring', 'Awareness', 'Grasp', 'Attempt'
    ]
    
    if text in seframes:
        text_labels = seframes[text]
        if any([elem in meaning_frames for elem in text_labels]):
            return True
        
    return False

In [10]:
fold_results = dict()
if os.path.isfile('bert_ds_android.json'):
    logger.info(Fore.YELLOW + "Loading data from cache" + Style.RESET_ALL)
    with open('bert_ds_android.json') as input_file:
        fold_results = json.load(input_file)

Loading data from cache


In [11]:
# @title Set environment variables

model_id = 'bert-base-uncased'
# model_id = 'distilbert-base-uncased'

import os
import contextlib
import tensorflow as tf
import os
import codecs
import numpy as np
import math
import json

import numpy as np
import pandas as pd

from collections import defaultdict, Counter
from tqdm import tqdm

USE_TPU = False
os.environ['TF_KERAS'] = '1'

# @title Initialize TPU Strategy
if USE_TPU:
    TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)
    tf.contrib.distribute.initialize_tpu_system(resolver)
    strategy = tf.contrib.distribute.TPUStrategy(resolver)

# sklearn libs
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

# Tensorflow Imports
import tensorflow as tf
from tensorflow.python import keras
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers


# Hugging face imports
from transformers import AutoTokenizer
from transformers import TFDistilBertForSequenceClassification, TFBertForSequenceClassification
from transformers import TFDistilBertModel, DistilBertConfig
from transformers import DistilBertTokenizerFast, BertTokenizerFast

Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


In [12]:
# @title Model parameters

# Bert Model Constants
SEQ_LEN = 64 # 128
BATCH_SIZE = 64 # 64 32 larger batch size causes OOM errors
EPOCHS = 10 # 3 4
LR = 1e-5 # 2e-5

# 3e-4, 1e-4, 5e-5, 3e-5
# My own constants
# USE_FRAME_FILTERING = False
# UNDERSAMPLING = True
# N_UNDERSAMPLING = 2 # ratio of how many samples from 0-class, to 1-class, e.g.: 2:1
# USE_DS_SYNTHETIC = False

USE_FRAME_FILTERING = False
UNDERSAMPLING = True
N_UNDERSAMPLING = 2 # ratio of how many samples from 0-class, to 1-class, e.g.: 2:1
USE_DS_SYNTHETIC = False
MIN_W = 3

In [13]:
# @title JSON to dataframe helper functions
def undersample_df(df, n_times=3):
    class_0,class_1 = df.category_index.value_counts()
    c0 = df[df['category_index'] == 0]
    c1 = df[df['category_index'] == 1]
    df_0 = c0.sample(int(n_times * class_1))
    
    undersampled_df = pd.concat([df_0, c1],axis=0)
    return undersampled_df

def get_ds_synthetic_data(min_w=MIN_W):
    short_task = {
      "bugzilla": """How to query bugs using the custom fields with the Bugzilla REST API?""",
      "databases": """Which technology should be adopted for the database layer abstraction: Object/Relational Mapping (ORM) or a Java Database Connectivity API (JDBC)?""",
      "gpmdpu": """Can I bind the cmd key to the GPMDPU shortcuts?""",
      "lucene": """How does Lucene compute similarity scores for the BM25 similarity?""",
      "networking": """Which technology should be adopted for the notification system, Server-Sent Events (SSE) or WebSockets?""",
    }

    with open('relevance_corpus.json') as ipf:
        aux = json.load(ipf)
        raw_data = defaultdict(list)
        for d in aux:
            if d['task'] == 'yargs':
                continue

            raw_data['text'].append(d['text'])
            raw_data['question'].append(short_task[d['task']])
            raw_data['source'].append(d['source'])
            raw_data['category_index'].append(1 if d['weight'] > min_w else 0)
            raw_data['weights'].append(d['weight'] if d['weight'] > min_w else 0)
 
        data = pd.DataFrame.from_dict(raw_data)
        data = undersample_df(data, n_times=1)
        data = data.sample(frac=1).reset_index(drop=True)
      
    return data

def get_class_weights(y, smooth_factor=0, upper_bound=5.0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    clazz = {cls: float(majority / count) for cls, count in counter.items()}
    result = {}
    for key, value in clazz.items():
        if value > upper_bound:
            value = upper_bound
        
        result[key] = value
    return result

def add_raw_data(result, data):
    s = data['source']
    if 'docs.oracle' in s or 'developer.android' in s:
        source_type = 'api'
    elif 'stackoverflow.com' in s:
        source_type = 'so'
    elif 'github.com' in s:
        source_type = 'git'
    else:
        source_type = 'misc'
    
    result['text'].append(data['text'])
    result['question'].append(data['question'])
    result['source'].append(data['source'])
    result['category_index'].append(data['category_index'])
    result['weights'].append(data['weights'])
    result['source_type'].append(source_type)


In [14]:
# @title Tokenizer

print(model_id)
if model_id == 'distilbert-base-uncased':
    tokenizer = DistilBertTokenizerFast.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)
else:
    tokenizer = BertTokenizerFast.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)

bert-base-uncased


In [15]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [16]:
# @title data encoder

def _encode(tokenizer, dataframe, max_length=SEQ_LEN):
    
    seq_a = dataframe['text'].tolist()
    seq_b = dataframe['question'].tolist()
    
    return tokenizer(seq_a, seq_b, truncation=True, padding=True, max_length=max_length)

def to_one_hot_encoding(data, nb_classes = 2):
    targets = np.array([data]).reshape(-1)
    one_hot_targets = np.eye(nb_classes)[targets]
    return one_hot_targets    

In [17]:
# @title Metrics & Logging functions

from sklearn.metrics import classification_report

recommendation_metrics = defaultdict(list)
prediction_metrics = defaultdict(list)
api_metrics = defaultdict(list)
so_metrics = defaultdict(list)
git_metrics = defaultdict(list)
misc_metrics = defaultdict(list)

classification_report_lst = []
log_examples_lst = []
source_lst = []
venn_diagram_set = []

def aggregate_macro_metrics(store_at, precision, recall, fscore):   
    store_at['precision'].append(precision)
    store_at['recall'].append(recall)
    store_at['fscore'].append(fscore)
    
    
def aggregate_macro_source_metrics(precision, recall, fscore, source):
    s = source
    if 'docs.oracle' in s or 'developer.android' in s:
        aggregate_macro_metrics(api_metrics, precision, recall, fscore)
    elif 'stackoverflow.com' in s:
        aggregate_macro_metrics(so_metrics, precision, recall, fscore)
    elif 'github.com' in s:
        aggregate_macro_metrics(git_metrics, precision, recall, fscore)        
    elif  'github.com' not in s and 'docs.oracle' not in s and 'developer.android' not in s and 'stackoverflow.com' not in s:
        aggregate_macro_metrics(misc_metrics, precision, recall, fscore)
    

def aggregate_recommendation_metrics(store_at, k, precision_at_k, pyramid_precision_at_k):
    store_at['k'].append(k)
    store_at['precision'].append(precision_at_k)
    store_at['∆ precision'].append(pyramid_precision_at_k)
    
def aggregate_report_metrics(clz_report):
    relevant_label = str(1)
    if relevant_label in clz_report:
        for _key in ['precision', 'recall']:
            if _key in clz_report[relevant_label]:
                clz_report_lst[_key].append(clz_report[relevant_label][_key])    
                
def log_examples(task_title, source, text, pweights, y_predict, y_probs, k=10):
    # get the predicted prob at every index
    idx_probs = [(idx, y_predict[idx], y_probs[idx]) for idx, _ in enumerate(y_predict)]
    
    # filter probs for all indexes predicted as relevant  
    idx_probs = list(filter(lambda k: k[1] == 1, idx_probs))
    
    most_probable = sorted(idx_probs, key=lambda i: i[2], reverse=True)
    
    result = [idx for idx, _, _ in most_probable][:k]
    
    for idx in result:
        log_examples_lst.append((
            source, 
            task_title,
            pweights[idx],
            y_predict[idx],
            y_probs[idx],
            text[idx]
        ))
        
def log_venn_diagram(y_true, y_predicted, text):
    cnt = 0
    try:
        for _true, _predict, _t in zip(y_true, y_predicted, text):
            if _true == 1 and _predict == 1:
                cnt += 1
                venn_diagram_set.append(_t)
    except Exception as ex:
        logger.info(str(ex))
    logger.info(Fore.RED + str(cnt) + Style.RESET_ALL + " entries logged")

    
def avg_macro_metric_for(data):
    __precision = data['precision']
    __recall = data['recall']
    __fscore = data['fscore']

    return np.mean(__precision), np.mean(__recall), np.mean(__fscore)        

In [18]:
#@title Training procedures

def get_train_val_test(task_uid, size=0.9, undersample=False, aug=True, undersample_n=3):
    if not isinstance(task_uid, list):
        task_uid = [task_uid]
        
    train_data_raw = defaultdict(list)
    test_data_raw = defaultdict(list)
    
    for _data in tqdm(CORPUS):
        if _data['question'] in task_uid:
            add_raw_data(test_data_raw, _data)
        else:
            add_raw_data(train_data_raw, _data)
    
    train_val = pd.DataFrame.from_dict(train_data_raw)
    test = pd.DataFrame.from_dict(test_data_raw)
    
    # https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    #  randomize rows....    
    train_val = train_val.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    
    if undersample:
        train_val = undersample_df(train_val, n_times=undersample_n)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
        
    if aug:
        train_val = pd.concat([train_val, get_ds_synthetic_data()],axis=0)
        train_val = train_val.sample(frac=1).reset_index(drop=True)
    
    weights = get_class_weights(train_val['category_index'].tolist())
    
    train, val = train_test_split(
        train_val, 
        stratify=train_val['category_index'].tolist(), 
        train_size=size
    )
    
    return train, val, test, weights        

In [19]:
def update_predictions(task_title, text, y_predict, y_probs, relevant_class=1):
    result = []
    
    for _t, _y, _prob in zip(text, y_predict, y_probs):
        if _y == relevant_class:
            if has_meaningful_frame(_t):
                result.append(_y)
            else:
                result.append(0)
        else:
            result.append(_y)
    
    return result    

In [20]:
# @title Testing procedures

# https://medium.com/geekculture/hugging-face-distilbert-tensorflow-for-custom-text-classification-1ad4a49e26a7
def eval_model(model, test_data):
    preds = model.predict(test_data.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()      

    return res.argmax(axis=-1), res[:, 1]

def test_model(source, df_test, model, tokenizer, pos_filter=False):
    
    df_source = df_test[df_test["source"] == source]   
    task_title = df_source['question'].tolist()[0]
    text = df_source['text'].tolist()
    pweights = df_source['weights'].tolist()
    
    # Encode X_test
    test_encodings = _encode(tokenizer, df_source)
    test_labels = df_source['category_index'].tolist()
    
    test_dataset = tf.data.Dataset.from_tensor_slices((
        dict(test_encodings),
        test_labels
    ))
    
    y_true = [y.numpy() for x, y in test_dataset]
    y_predict, y_probs = eval_model(model, test_dataset)
    
    if pos_filter:
        y_predict = update_predictions(task_title, text, y_predict, y_probs)
    

    accuracy = accuracy_score(y_true, y_predict)
    macro_f1 = f1_score(y_true, y_predict, average='macro')
    
    classification_report_lst.append(classification_report(y_true, y_predict))
    aggregate_report_metrics(classification_report(y_true, y_predict, output_dict=True))
    

    logger.info("-" * 20)    
    
    logger.info("Y")
    logger.info("[0s] {} [1s] {}".format(
        len(list(filter(lambda k: k== 0, y_true))),
        len(list(filter(lambda k: k== 1, y_true)))
    ))
    
        
    logger.info("predicted")
    logger.info("[0s] {} [1s] {}".format(
        len(list(filter(lambda k: k== 0, y_predict))),
        len(list(filter(lambda k: k== 1, y_predict)))
    ))
    
    logger.info("-" * 20)
    
    logger.info("Accuracy: {:.4f}".format(accuracy))
    logger.info("macro_f1: {:.4f}".format(macro_f1))

    precision, recall, fscore, _ = precision_recall_fscore_support(y_true, y_predict, average='macro')
    
    aggregate_macro_metrics(prediction_metrics, precision, recall, fscore)
    aggregate_macro_source_metrics(precision, recall, fscore, source)
    
    logger.info("Precision: {:.4f}".format(precision))
    logger.info("Recall: {:.4f}".format(recall))
    logger.info("F1: {:.4f}".format(fscore))
    
    log_examples(task_title, source, text, pweights, y_predict, y_probs, k=10)
    log_venn_diagram(y_true, y_predict, text)
    source_lst.append(source)

In [21]:
def add_idx_fold_results(idx_split, store_at):
    if idx_split not in store_at:
        store_at[idx_split] = dict()
        store_at[idx_split]['run_cnt'] = 0
        store_at[idx_split]['overall'] = defaultdict(list)
        store_at[idx_split]['api'] = defaultdict(list)
        store_at[idx_split]['so'] = defaultdict(list)
        store_at[idx_split]['git'] = defaultdict(list)
        store_at[idx_split]['misc'] = defaultdict(list)
    
    store_at[idx_split]['run_cnt'] += 1
    
    _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)
    store_at[idx_split]['overall']['precision'].append(_precision)
    store_at[idx_split]['overall']['recall'].append(_recall)
    store_at[idx_split]['overall']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(api_metrics)
    store_at[idx_split]['api']['precision'].append(_precision)
    store_at[idx_split]['api']['recall'].append(_recall)
    store_at[idx_split]['api']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(so_metrics)
    store_at[idx_split]['so']['precision'].append(_precision)
    store_at[idx_split]['so']['recall'].append(_recall)
    store_at[idx_split]['so']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(git_metrics)
    store_at[idx_split]['git']['precision'].append(_precision)
    store_at[idx_split]['git']['recall'].append(_recall)
    store_at[idx_split]['git']['fscore'].append(_f1score)  
    
    _precision, _recall, _f1score = avg_macro_metric_for(misc_metrics)
    store_at[idx_split]['misc']['precision'].append(_precision)
    store_at[idx_split]['misc']['recall'].append(_recall)
    store_at[idx_split]['misc']['fscore'].append(_f1score)  

In [22]:
# model = TFBertForSequenceClassification.from_pretrained(model_id, cache_dir='/home/msarthur/scratch', local_files_only=True)

In [23]:
# @title 10-fold cross validation WIP
CORPUS = raw_data

all_tasks = sorted(list(set([d['question'] for d in raw_data])))
rseed = 20210343
random.seed(rseed)
random.shuffle(all_tasks)

from sklearn.model_selection import KFold


file_handler = logging.FileHandler('/home/msarthur/scratch/LOG-bert_ds_android.ans')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)


n_splits = 10
kf = KFold(n_splits=n_splits, random_state=rseed)
np_tasks_arr = np.array(all_tasks)



idx_split = 0
for train_index, test_index in kf.split(np_tasks_arr):

    idx_split = str(idx_split)
    eval_fold = True
    # 10 runs per fold to avoid reporting peek results in a given fold
    if idx_split in fold_results and fold_results[idx_split]['run_cnt'] >= 10:
        logger.info(Fore.RED + f"Fold {idx_split} FULLY TESTED" + Style.RESET_ALL)
        eval_fold = False


    if eval_fold:
        # <------------------------------------------------------------------------- EVAL VARIABLES
        recommendation_metrics = defaultdict(list)
        prediction_metrics = defaultdict(list)
        api_metrics = defaultdict(list)
        so_metrics = defaultdict(list)
        git_metrics = defaultdict(list)
        misc_metrics = defaultdict(list)
        random_prediction_metrics = defaultdict(list)
        clz_report_lst = defaultdict(list)

        classification_report_lst = []
        log_examples_lst = []
        source_lst = []
        venn_diagram_set = []
        # <------------------------------------------------------------------------- EVAL VARIABLES


        test_tasks_lst = np_tasks_arr[test_index].tolist()

        logger.info("")
        logger.info(Fore.RED + f"Fold {idx_split}" + Style.RESET_ALL)
        logger.info('\n'.join(test_tasks_lst))

        # <------------------------------------------------------------------------- INPUT
        df_train, df_val, df_test, weights = get_train_val_test(
            test_tasks_lst,
            aug=USE_DS_SYNTHETIC,
            undersample=UNDERSAMPLING, 
            undersample_n=N_UNDERSAMPLING
        )
        # <------------------------------------------------------------------------- INPUT

        logger.info('-' * 10)
        logger.info(Fore.RED + 'train'+ Style.RESET_ALL)
        logger.info(str(df_train.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'test'+ Style.RESET_ALL)
        logger.info(str(df_test.category_index.value_counts()))
        logger.info("")

        logger.info(Fore.RED + 'weights'+ Style.RESET_ALL)
        logger.info(str(weights))
        logger.info('-' * 10)


        # Encode X_train
        train_encodings = _encode(tokenizer, df_train)
        train_labels = df_train['category_index'].tolist()

        # Encode X_valid
        val_encodings = _encode(tokenizer, df_val)
        val_labels = df_val['category_index'].tolist()


        # https://huggingface.co/transformers/custom_datasets.html
        train_dataset = tf.data.Dataset.from_tensor_slices((
            dict(train_encodings),
            train_labels
        ))

        val_dataset = tf.data.Dataset.from_tensor_slices((
            dict(val_encodings),
            val_labels
        ))


        if model_id == 'distilbert-base-uncased':
            model = TFDistilBertForSequenceClassification.from_pretrained(
                model_id, cache_dir='/home/msarthur/scratch'
            )
        else:
            model = TFBertForSequenceClassification.from_pretrained(
                model_id, cache_dir='/home/msarthur/scratch', local_files_only=True
            )

        # freeze all the parameters
        # for param in model.parameters():
        #   param.requires_grad = False


        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

        METRICS = [
            tf.keras.metrics.SparseCategoricalAccuracy()
        ]

        early_stopper = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', mode='min', patience=4, 
            verbose=1, restore_best_weights=True
        )

        # https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint
        checkpoint_filepath = '/home/msarthur/scratch/best_model'

        mc = tf.keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, 
            monitor='val_loss', mode='min', verbose=1, 
            save_best_only=True,
            save_weights_only=True
        )

        model.compile(
            optimizer=optimizer,
            loss=loss_fn,
            metrics=METRICS
        )

        # https://discuss.huggingface.co/t/how-to-dealing-with-data-imbalance/393/3
        # https://wandb.ai/ayush-thakur/huggingface/reports/Early-Stopping-in-HuggingFace-Examples--Vmlldzo0MzE2MTM
        model.fit(
            train_dataset.shuffle(1000).batch(BATCH_SIZE), 
            epochs=EPOCHS, 
            batch_size=BATCH_SIZE,
            class_weight=weights,
            validation_data=val_dataset.shuffle(1000).batch(BATCH_SIZE),
            callbacks=[early_stopper, mc]
        )

        model.load_weights(checkpoint_filepath)

        logger.info("")
        logger.info(Fore.RED + f"Testing model" + Style.RESET_ALL)
        for source in df_test["source"].unique():
            df_source = df_test[df_test["source"] == source]   
            logger.info(source)
            test_model(source, df_source, model, tokenizer, pos_filter=USE_FRAME_FILTERING)

        add_idx_fold_results(idx_split, fold_results)
        if 'venn_diagram_set' not in fold_results:
            fold_results['venn_diagram_set'] = []

        fold_results['venn_diagram_set'] += venn_diagram_set
        fold_results['venn_diagram_set'] = list(set(fold_results['venn_diagram_set']))


        _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)

        logger.info("")
        logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
        logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
        logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
        logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)




        log_sources_data = [api_metrics, so_metrics, git_metrics, misc_metrics]
        log_sources_ids = ['api_metrics', 'so_metrics', 'git_metrics', 'misc_metrics']

        for _id, __data in zip(log_sources_ids, log_sources_data):
            _precision, _recall, _f1score = avg_macro_metric_for(__data)

            logger.info("")
            logger.info(Fore.YELLOW + f"{_id}" + Style.RESET_ALL)
            logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
            logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
            logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


    idx_split = int(idx_split)
    idx_split += 1
    logger.info(f"next {idx_split}")
#     break
#         if idx_split >= 7:
#             logger.info(f"breaking at {idx_split}")
#             break


Fold 0
how can i get the value of text view in recyclerview item?
Hide MarkerView when nothing selected
How to check programmatically whether app is running in debug mode or not?
JSONObject parse dictionary objects
Want to add drawable icons insteadof colorful dots


/home/msarthur/hface/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
100%|██████████| 7925/7925 [00:00<00:00, 662452.10it/s]

----------
train
0    1661
1     831
Name: category_index, dtype: int64

test
0    669
1     66
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 2.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x2b2143e413d0>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expe

...

100%|██████████| 7925/7925 [00:00<00:00, 328567.50it/s]

----------
train
0    1627
1     813
Name: category_index, dtype: int64

test
0    493
1     85
Name: category_index, dtype: int64

weights
{0: 1.0, 1: 2.0}
----------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
39/39 [==============================] - ETA: 0s - loss: 0.9258 - sparse_categorical_accuracy: 0.5533The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output

recall:    0.647
f1-score:  0.572

api_metrics
precision: 0.556
recall:    0.600
f1-score:  0.448

so_metrics
precision: 0.680
recall:    0.699
f1-score:  0.642

git_metrics
precision: 0.608
recall:    0.810
f1-score:  0.639

misc_metrics
precision: 0.547
recall:    0.555
f1-score:  0.538
next 10


In [24]:
__precision, __recall, __fscore = [], [], []

for key_i, value in fold_results.items():
    if isinstance(value, dict):
        for key_j, __data in value.items():
            if key_j == 'overall':
                logger.info(Fore.YELLOW + f"{key_i}" + Style.RESET_ALL)
                logger.info("precision: " + Fore.RED +
                            "{:.3f}".format(np.mean(__data['precision'])) + Style.RESET_ALL +
                           f" {str([round(x, 2) for x in __data['precision']])}")
                logger.info("recall:    " + Fore.RED +
                            "{:.3f}".format(np.mean(__data['recall'])) + Style.RESET_ALL+
                           f" {str([round(x, 2) for x in __data['recall']])}")
                logger.info("f1-score:  " + 
                            Fore.RED + "{:.3f}".format(np.mean(__data['fscore'])) + Style.RESET_ALL+
                           f" {str([round(x, 2) for x in __data['fscore']])}")
                
                __precision += __data['precision']
                __recall += __data['recall']
                __fscore += __data['fscore']
                
__precision = [x for x in __precision if str(x) != 'nan']
__recall = [x for x in __recall if str(x) != 'nan']
__fscore = [x for x in __fscore if str(x) != 'nan']


logger.info("\n")
logger.info(Fore.RED + "AGGREGATED METRICS" + Style.RESET_ALL)
logger.info("\nprecision: " + Fore.RED + "{:.3f}".format(np.mean(__precision)) + Style.RESET_ALL)
logger.info("recall:    " + Fore.RED + "{:.3f}".format(np.mean(__recall)) + Style.RESET_ALL)
logger.info("f1-score:  " +  Fore.RED + "{:.3f}".format(np.mean(__fscore)) + Style.RESET_ALL)

0
precision: 0.548 [0.57, 0.58, 0.52, 0.54, 0.56, 0.54, 0.53, 0.53, 0.55]
recall:    0.569 [0.59, 0.61, 0.56, 0.56, 0.58, 0.55, 0.55, 0.55, 0.56]
f1-score:  0.498 [0.55, 0.53, 0.46, 0.52, 0.52, 0.48, 0.47, 0.48, 0.47]
1
precision: 0.612 [0.64, 0.61, 0.55, 0.64, 0.59, 0.63, 0.63, 0.58, 0.65]
recall:    0.626 [0.66, 0.66, 0.55, 0.65, 0.59, 0.64, 0.64, 0.59, 0.66]
f1-score:  0.531 [0.55, 0.54, 0.51, 0.49, 0.54, 0.53, 0.54, 0.51, 0.56]
2
precision: 0.548 [0.56, 0.47, 0.57, 0.56, 0.55, 0.54, 0.56, 0.57, 0.57]
recall:    0.574 [0.6, 0.5, 0.58, 0.59, 0.59, 0.55, 0.55, 0.6, 0.6]
f1-score:  0.487 [0.5, 0.46, 0.45, 0.46, 0.52, 0.51, 0.51, 0.47, 0.5]
3
precision: 0.510 [0.5, 0.54, 0.5, 0.53, 0.53, 0.5, 0.53, 0.46, 0.5]
recall:    0.554 [0.55, 0.59, 0.54, 0.58, 0.57, 0.54, 0.56, 0.52, 0.54]
f1-score:  0.434 [0.44, 0.47, 0.45, 0.48, 0.41, 0.42, 0.43, 0.43, 0.39]
4
precision: 0.569 [0.57, 0.52, 0.55, 0.58, 0.52, 0.54, 0.57, 0.69, 0.58]
recall:    0.598 [0.6, 0.54, 0.57, 0.61, 0.61, 0.58, 0.6, 0.64, 

In [25]:
logger.info(Fore.YELLOW + "Caching results" + Style.RESET_ALL)
with open('bert_ds_android.json', 'w') as fo:
    json.dump(fold_results, fo, indent=4)

Caching results


In [26]:
fold_results.keys()

dict_keys(['0', 'venn_diagram_set', '1', '2', '3', '4', '5', '6', '7', '8', '9'])

In [27]:
# cnt = 0
# for source in df_test["source"].unique():
#     df_source = df_test[df_test["source"] == source]   
#     logger.info(source)
#     test_model(source, df_source, model, tokenizer, pos_filter=True)
#     cnt += 1
#     if cnt >= 5:
#         break

In [28]:
#@title Metrics report
# logger.info(json.dumps(fold_results, indent=4, sort_keys=True))

In [29]:
# _precision, _recall, _f1score = avg_macro_metric_for(prediction_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "Model metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)


# _precision, _recall, _f1score = avg_macro_metric_for(api_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "API metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(so_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "SO metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(git_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "GIT metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

# _precision, _recall, _f1score = avg_macro_metric_for(misc_metrics)

# logger.info("")
# logger.info(Fore.YELLOW + "MISC metrics" + Style.RESET_ALL)
# logger.info("precision: " + Fore.RED + "{:.3f}".format(_precision) + Style.RESET_ALL)
# logger.info("recall:    " + Fore.RED + "{:.3f}".format(_recall) + Style.RESET_ALL)
# logger.info("f1-score:  " + Fore.RED + "{:.3f}".format(_f1score) + Style.RESET_ALL)

In [30]:
def examples_per_source_type(source_type='misc', n_samples=None):
    _sources = list(set([x[0] for x in log_examples_lst]))

    _template = "[w={}]" + Fore.RED + "[y={}]" + Fore.YELLOW + "[p={:.4f}]" + Style.RESET_ALL + " {}"

    idx = 0
    for s in _sources:
        examples_in_source = []
        if source_type == 'api' and ('docs.oracle' in s or 'developer.android' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        elif source_type == 'so' and ('stackoverflow.com' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]            
            idx += 1
        elif source_type == 'git' and ('github.com' in s):
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        elif source_type == 'misc' and 'github.com' not in s and 'docs.oracle' not in s and 'developer.android' not in s and 'stackoverflow.com' not in s:
            examples_in_source = list(filter(lambda k: k[0] == s, log_examples_lst))
            task_title = examples_in_source[0][1]
            idx += 1
        if not examples_in_source:
            continue
        logger.info('')
        logger.info(Fore.RED + f"{task_title}" + Style.RESET_ALL)    
        logger.info(s)
        logger.info('')

        for _, _, pweights, y_predict, y_probs, text in examples_in_source:
            logger.info(_template.format(pweights, y_predict, y_probs, text))
            logger.info('')
        logger.info('-' * 20)
      
        if n_samples and idx >= n_samples:
            break
    

In [31]:
#@title Sample prediction outputs for API sources

logger.info(Fore.RED + "API" + Style.RESET_ALL)
examples_per_source_type(source_type='api', n_samples=8)

API

Hilt: How to prevent Hilt from picking dependency from a library?
https://developer.android.com/training/dependency-injection/hilt-android

[w=0][y=1][p=0.9658] Each Hilt component is responsible for injecting its bindings into the corresponding Android class.

[w=0][y=1][p=0.9638] Hilt automatically creates and destroys instances of generated component classes following the lifecycle of the corresponding Android classes.

[w=0][y=1][p=0.9577] The parameters of an annotated constructor of a class are the dependencies of that class.

[w=0][y=1][p=0.9563] Doing manual dependency injection requires you to construct every class and its dependencies by hand, and to use containers to reuse and manage dependencies.

[w=0][y=1][p=0.9553] A binding contains the information necessary to provide instances of a type as a dependency.

[w=0][y=1][p=0.9545] Hilt only creates a scoped binding once per instance of the component that the binding is scoped to, and all requests for that binding share

In [32]:
#@title Sample prediction outputs for GIT sources

logger.info(Fore.RED + "GIT" + Style.RESET_ALL)
examples_per_source_type(source_type='git', n_samples=8)

GIT

Hilt: How to prevent Hilt from picking dependency from a library?
https://github.com/google/dagger/issues/1991

[w=0][y=1][p=0.9583] The application gradle module only contains your Application class and defines the root of your dependency injection graph.

[w=0][y=1][p=0.8941] You then compose your application by pulling in the necessary dependencies.

[w=1][y=1][p=0.8647] Instead of changing implementation to api ( Not Recommended ), you can add dependencies of missing libraries in app's build.gradle and rebuild the project.

[w=0][y=1][p=0.8541] There are still great benefits to making separate gradle modules, for one you can have smaller tests on them, including ` small test APKs' where the dependencies set and the modules in the classpath are reduced.

[w=0][y=1][p=0.8165] In this project all feature/module projects declare dependencies on their api and impl using:

[w=0][y=1][p=0.8066] The difference in Hilt is that it can be easier to encounter since modules are aggregated 

In [33]:
#@title Sample prediction outputs for SO sources

logger.info(Fore.RED + "SO" + Style.RESET_ALL)
examples_per_source_type(source_type='so', n_samples=8)

SO

Android SQLite performance in complex queries
https://stackoverflow.com/questions/4015026

[w=0][y=1][p=0.9102] You can have indexes that contain multiple columns -LRB- to assist queries with multiple predicates -RRB-.

[w=1][y=1][p=0.8903] If you use an ordering predicate -LRB- <, < =, > etc -RRB- then that needs to be in the last used column of the index.

[w=3][y=1][p=0.8879] For SELECTs and UPDATEs, indexes can things up, but only if the indexes you create can actually be used by the queries that you need speeding up.

[w=0][y=1][p=0.8468] LINK -LRB- of table structures -RRB- is also worth considering -LRB- if you haven't already -RRB- simply because it tends to provide the smallest representation of the data in the database ; this is a trade-off, less I/O for more CPU, and one that is usually worthwhile in medium-scale enterprise databases -LRB- the sort I'm most familiar with -RRB-, but I'm afraid I've no idea whether the trade-off works well on small-scale platforms like And

In [34]:
#@title Sample prediction outputs for MISC sources

logger.info(Fore.RED + "MISC" + Style.RESET_ALL)
examples_per_source_type(source_type='misc', n_samples=8)

MISC

Hilt: How to prevent Hilt from picking dependency from a library?
https://prog.world/a-practical-guide-to-using-hilt-with-kotlin

[w=0][y=1][p=0.9527] In some cases, you will have a class that is not is created by dependency injection, but has dependencies injected into it.

[w=0][y=1][p=0.9457] ⮕ Adding Annotation to the Designer @Inject Any class can have an annotated constructor @Inject, allowing it to be used as a dependency anywhere in the project.

[w=0][y=1][p=0.9315] ⮕ As constructor parameters If you mark the constructor with annotation @Inject, Hilt will implement all the parameters according to the bindings you define for these types.

[w=1][y=1][p=0.9266] They are not mutually exclusive: in many cases, a class is both injectable and has dependencies.

[w=0][y=1][p=0.9158] null ... Note that dependencies should be injected as fields only when the class must have a parameterless constructor, for example Activity ... In most cases, we recommend that you inject dependenci

In [35]:
logger.info(Fore.RED + f"{len(fold_results['venn_diagram_set'])} entries VENN SET" + Style.RESET_ALL)
for _t in fold_results['venn_diagram_set']:
    logger.info(_t)

881 entries VENN SET

Since the version of Retrofit 2.6.0, you can get the annotations in OkHttp Interceptor using the tag field like this:
Typically, the computed size is just big enough for the control and the label to be fully visible.
ComponentActivity, the base class for FragmentActivity and AppCompatActivity, allows you to control the behavior of the Back button by using its OnBackPressedDispatcher, which you can retrieve by calling getOnBackPressedDispatcher ( ).
Call ActivityOptions.setLockTaskEnabled ( ) and supply these options when starting the activity.
get a seekable file descriptor from your pdf document:
If you want to set a specific size for your camera preview, set this in the surfaceChanged ( ) method as noted in the comments above.
Of the suggestions proposed, LINK actually combines observable results with each other, which may or may not be what is wanted, but was not asked in the question.
Make a Snackbar to display a message.
These permission can then be allowed o

After some time, I figured out, that it works if I annotate the stations field with a @JsonProperty annotation:
There are three ways to define anchor in Hilt.
That is because, when your app starts, the ViewPager displays the movie at index 0.
gravity is the way the text will align itself in the TextView.
We instantiate a new LocationRequest object.
Getting a bitmap cache from a layout or a view by doing something like First you got ta setDrawingCacheEnabled to a layout -LRB- a linearlayout or relativelayout, or a view -RRB-
The easiest way to simulate network issues with MockWebServer is by setting the SocketPolicy to SocketPolicy.DISCONNECT _ AT_START, SocketPolicy.NO _ RESPONSE or etc:
I run into the same issue today ( I suppose ).
You may extend it from BaseAdapter.
Using both WHERE predicates and ORDER BY both require an index and SQLite can only use one, so that can be a point where performance suffers.
In MainActivity, add the EXTRA_MESSAGE constant and the sendMessage ( ) code, 

If the user granted the permission to your app, you can access the private user data.
You can read more about this on the material design guidelines which state: `` The use of application icon plus title as a standard layout is discouraged on API 21 devices and newer.''
This line means: add a meta-property on serialization or read a meta-property on deserialization -LRB- include = JsonTypeInfo.As.PROPERTY -RRB- called'' @class'' -LRB- property ='' @class'' -RRB- that holds the fully-qualified Java class name -LRB- use = JsonTypeInfo.Id.CLASS -RRB-.
The ArrayAdapter fits in between an ArrayList ( data source ) and the ListView ( visual representation ) and configures two aspects:
When a fragment is not visible, the PagerAdapter will detach it, but not destroy it, so the fragment instance remains alive in the FragmentManager.
Normal Permissions When you need to add a new permission, first check this page to see if the permission is considered a PROTECTION_NORMAL permission.
You may find 

Fragment Hiding vs Replace In many of the examples above, we call transaction.replace ( ... ) to load a dynamic fragment which first removes the existing fragment from the activity invoking onStop and onDestroy for that fragment before adding the new fragment to the container.
In Dagger 2, you can define how components should be encapsulated by defining custom scopes.
Open MoviesPagerAdapter.kt and add the following method inside the class:
First, define the qualifiers that you will use to annotate the @Binds or @Provides methods:
To learn how to validate the user's response token, see Verifying the user's response.
Now make your activity implement the interface:
Anyone with HTTP POST knowledge could put random data inside of the g-recaptcha-response form field, and foll your site to make it think that this field was provided by the google widget.
Capture and Save Files - Setup the code for capturing pictures or videos and saving the output.
There are two types of standard PagerAdapter

Tap on `` Clear Cache'' button to flush the cache.
Returns the value mapped by name if it exists and is a JSONObject, or throws otherwise.
You can see that it returns an application instance using PotterTestApp instead of PotterApp.
However, on rooted devices an application can obtain root access and read everything.
If we need two different objects of the same return type, we can use the @Named qualifier annotation.
It works kind of like this: The ListView asks the adapter what it should display, and the adapter jumps into action:
Pull the data all at once - not in small parts.
Make sure you add it outside of the application tag.
Use this to benchmark any gains as you optimize.
Finally, getView ( ) creates a view to be used as a row in the list.
one way could be to encrypt it using a passcode.
For simplicity in rendering a camera preview, you should change your application's preview activity orientation to landscape by adding the following to your manifest.
Dagger 2 analyzes these dep

...

However, there are a few cases where your app might need to implement its own Back behavior in order to provide the best possible user experience.
To do this, we might modify this code:
Have you tried Jackson ?
LINK allows filtering collections without writing loops or inner classes:
Usage Defining a Fragment A fragment, like an activity, has an XML layout file and a Java class that represents the Fragment controller.
Currently the recommended API for making screenshots is the LINK class available from API 24 -LRB- but the methods which accept Window parameter are available from API 26 == Android 8.0 Oreo -RRB-.
Next, we will create a BufferedReader that will allow us to iterate through the response.
Do not inset the content with any margins from the PrintAttributes as the application is responsible to render it such that the margins are respected.
There, you can define dependencies with the @Provides annotation.
When a list item is scrolled off the screen, RecyclerView reuses that vie

Such instructions are called bindings ...
All I did was create a counter for the number of calls I'm going to do, then each async result calls ready -LRB- -RRB- -LRB- be sure to do this on the failure methods too, unless you're going to do something different -RRB-
Let's create another function called makeLoginRequest ( ) that moves the execution to the background thread and ignores the response for now:
I would suggest that you write your downloaded JSON out to a file and compare it with your original to see if there is a problem with the download logic.
Here's how you might use it to swipe across a collection of Fragment objects:
In earlier versions of Android, accepting potentially dangerous permission groups was an all-or-nothing affair.
You are using the AppCompat version 21 + and it is normal.
the second step is to translate the view up and left by half the width and half the height.
( what I could not convert with native-image using the entire tree, yet works when you pick more 

The adapter acts as the middle man between the ListView and data source, or its provider.
We didn't say that there should many sub-streams running concurrently.
Although the icon can be added back with:
The app will not crash, since all the
Get relevant subviews of the row view.
The TextView being in wrap_content this does nothing, as the TextView is exactly the size of the text.
Returns the value mapped by name, or throws if no such mapping exists.
Returns the value mapped by name if it exists and is an int or can be coerced to an int, or 0 otherwise.
I think your DataSource is implemented incorrectly.
Beginning with Android 5.0, notifications can appear on the lock screen.
Decide whether you trust this provider more than the one used in your app's cached location.
So no need to implement your async tasks.
You can define multiple bindings for the same type with qualifiers.
Returns the value mapped by name if it exists and is an int or can be coerced to an int, or fallback otherwise.
F

Nothing much happening here before starting the test we create the server and start it at port 8080.
The reason why dataSpec.position needs to be included is already explained as clearly as I'm able to explain it in my response above.
Must be one of the following constant values.
To be able to record, your app must tell the user that it will access the device's audio input.
The use of application icon plus title as a standard layout is discouraged on API 21 devices and newer.
The system displays a runtime permission prompt, such as the one shown on the permissions overview page.
The function makeLoginRequest ( ) should take a callback as a parameter so that it can return a value asynchronously.
Users should know which actions might require them to grant permission for your app to access private user data.
First function of scopes is a way to tell Dagger compiler which scopes are allowed within scoped component.
Your next step is to implement the adapter methods.
Here's a bit of code to

For SELECTs and UPDATEs, indexes can things up, but only if the indexes you create can actually be used by the queries that you need speeding up.
Returns the value mapped by name if it exists and is a double or can be coerced to a double, or fallback otherwise.
For each of the get methods, you must pass the index position of the column you desire, which you can get by calling getColumnIndex ( ) or getColumnIndexOrThrow ( ).
Both methods have the same return type, but the qualifiers label them as two different bindings:
Returns the value mapped by name if it exists and is a long or can be coerced to a long, or throws otherwise.
Then, Hilt needs to know how to provide an instance of the type that corresponds with each qualifier.
Here is full code for my CustomViewPager:
After that, extend the class AsyncTask.
To set this up, you'll need a mechanism to tell the app to use the real URL normally, but the mock URL when you run tests.
I simply reorder the fragment list, update the viewPager a